# Scraping tweets related to COVID-19 in Brazil

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import tweepy as tw
import matplotlib.pyplot as plt
import re
import datetime
import nltk

In [340]:
# Twitter developer credentials
with open('twitter_key.txt', 'r') as kfile:
    consumer_key = kfile.readline().strip('\n')
    consumer_secret = kfile.readline().strip('\n')
    access_token = kfile.readline().strip('\n')
    access_token_secret = kfile.readline().strip('\n')

In [341]:
# Authentication - keys and tokens
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [383]:
# Define main searchwords to look inside twitter
search_words = ["coronavirus -filter:retweets",
                "corona -filter:retweets",
                "'Vacina de Oxford' -filter:retweets",
                "pandemia -filter:retweets",
                "covid19 -filter:retweets",
                "covid -filter:retweets",
                "covid-19 -filter:retweets",
                "gripezinha -filter:retweets",
                "#BrasilNaoPodeParar -filter:retweets"]

result_type = ['popular']

max_tweets = 600
tweets_df = pd.DataFrame([])

# Script to scrape data from Twitter

for word in range(len(search_words)):
    tweets = tw.Cursor(api.search, q=search_words[word],
                       result_type=result_type,lang='pt',count=100,
                       tweet_mode = 'extended',since='2020-10-26', until = '2020-10-28').items(max_tweets)
    
    tweets_list = [[tweet.full_text, tweet.created_at] for tweet in tweets]
    
    tweets_df = tweets_df.append(pd.DataFrame(tweets_list, columns=['text','created_at']), ignore_index=True)

In [384]:
tweets_df1 = tweets_df.copy()
tweets_df1

,text,created_at
0,"@roxmo Se você falasse em identificar pessoas com DSTs para manter as que não têm seguras, seria taxado de nazista na hora. Agora, com o Coronavirus Chinês, os comunistas logo mostram a que vieram.",2020-10-27 23:59:36
1,"@hxyazptt @guztavoliveira A energia criativa pensando na sua fantasia e ela é composta de uma sunga e um papelão e glitter, a energia de corpos dançando, se tocando, se vendo, se desejando, se soltando e se permitindo dançar, é sobre isso o fervo.\nÉ isso que você me tirou coronavirus, recarga.",2020-10-27 23:59:27
2,O jovem de hoje teme apenas uma coisa. Coronavirus ? Não... O Lisca Doido 🤡,2020-10-27 23:59:23
3,"Como são as coisas né, eu andado de máscara na pela rua, da aparência de eu é que estou errado e todo eles certos em não usar suas máscaras!\n#COVID19 \n#coronavirus",2020-10-27 23:59:01
4,"Rumo ao fundo do poço. \nDívida pública federal passa de R$ 4,5 tri com custo mais caro e prazo menor\nTesouro vê investidor preocupado com coronavírus e política econômica brasileira.\n https://t.co/DyWnClCTAa",2020-10-27 23:58:30
...,...,...
4795,@CNNBrasil Gripezinha,2020-10-26 10:59:19
4796,"@GloboNews Este Dr. em fev/20 disse que a gripe chinesa era uma ""gripezinha"" , em mar/20 ele pediu para ""ficar em casa "" e em set/20 ainda com a pandemia ele pede para a população sair de casa e ser mesário na eleição, o Brasil realmente não é para amadores !!!!!!!!!!!",2020-10-26 10:52:04
4797,Eá rinfo de quê Bolso Nero com 147 mil mortos pela gripezinha?,2020-10-26 10:51:06
4798,@____W00dst0ck__ Se tornou uma gripezinha.,2020-10-26 10:48:13


# Pre-processing

In [385]:
# Text to lowercase
lower_tweets = tweets_df1['text'].str.lower()
tweets_df1['text'] = lower_tweets

# substitute abbreviations with full words
ids = {'id':['pq ','vc ','vcs ','tb ','add ','vlw ','kd ','abs ','tc ','qdo ','qd ',
             'msg ','blz ','q ','ki ','dps ','dpois ','qto ','qt ','qts ',
            'td ','tds ','naum ','rs ','n ','entaum ','\bd\b','obs ','niver ','bjo ','bj ',
            'pc ','aki ','aq ','cmg ','mto ','mt ','pra ','agr ','akilo '],
      'full_word':['por que ','você ','vocês ','também ','adicionar ','valeu ','cadê ','abraço ','teclar ','quando ','quando ',
              'mensagem ','beleza ','que ','que ','depois ','depois ','quanto ','quanto ','quantos ',
             'tudo ','todos ','não ','risos ','não ','então ','de ','observação ','aniversário ','beijo ','beijo ',
             'computador ','aqui ','aqui ','comigo ','muito ','muito ','para ','agora ','aquilo ']}

ids = dict(zip(ids['id'], ids['full_word']))

repl = {rf'\b{word_id}\b': word for word_id, word in ids.items()}
cleaned_tweets = tweets_df1.replace({'text': repl}, regex=True)

tweets_df1['text'] = cleaned_tweets
tweets_df1

,text,created_at
0,"@roxmo se você falasse em identificar pessoas com dsts para manter as que não têm seguras, seria taxado de nazista na hora. agora, com o coronavirus chinês, os comunistas logo mostram a que vieram.",2020-10-27 23:59:36
1,"@hxyazptt @guztavoliveira a energia criativa pensando na sua fantasia e ela é composta de uma sunga e um papelão e glitter, a energia de corpos dançando, se tocando, se vendo, se desejando, se soltando e se permitindo dançar, é sobre isso o fervo.\né isso que você me tirou coronavirus, recarga.",2020-10-27 23:59:27
2,o jovem de hoje teme apenas uma coisa. coronavirus ? não... o lisca doido 🤡,2020-10-27 23:59:23
3,"como são as coisas né, eu andado de máscara na pela rua, da aparência de eu é que estou errado e todo eles certos em não usar suas máscaras!\n#covid19 \n#coronavirus",2020-10-27 23:59:01
4,"rumo ao fundo do poço. \ndívida pública federal passa de r$ 4,5 tri com custo mais caro e prazo menor\ntesouro vê investidor preocupado com coronavírus e política econômica brasileira.\n https://t.co/dywnclctaa",2020-10-27 23:58:30
...,...,...
4795,@cnnbrasil gripezinha,2020-10-26 10:59:19
4796,"@globonews este dr. em fev/20 disse que a gripe chinesa era uma ""gripezinha"" , em mar/20 ele pediu para ""ficar em casa "" e em set/20 ainda com a pandemia ele pede para a população sair de casa e ser mesário na eleição, o brasil realmente não é para amadores !!!!!!!!!!!",2020-10-26 10:52:04
4797,eá rinfo de quê bolso nero com 147 mil mortos pela gripezinha?,2020-10-26 10:51:06
4798,@____w00dst0ck__ se tornou uma gripezinha.,2020-10-26 10:48:13


In [386]:
# Removing @names
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text

# We are keeping cleaned tweets in a new column called 'tidy_tweets'
tweets_df1['text'] = np.vectorize(remove_pattern)(tweets_df1['text'], "@[\w]*")

In [387]:
# Removing Links
cleaned_tweets = []

for index, row in tweets_df1.iterrows():
    # Filtering out all the words that contains link
    no_links = [word for word in row.text.split() if 'http' not in word]
    cleaned_tweets.append(' '.join(no_links))

tweets_df1['text'] = cleaned_tweets

In [ ]:
# Save dataframe on excel format
#tweets_df1.to_excel("tweets_raw.xlsx")